<a href="https://colab.research.google.com/github/ncduy/catboy/blob/main/sell10contract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Please change the following to your own PAPER api key and secret
# or set them as environment variables (ALPACA_API_KEY, ALPACA_SECRET_KEY).
# You can get them from https://alpaca.markets/

api_key = "PKAJRW6L76J4ZWL5G61L"
secret_key = "9Ac6q53gbgEEEXvxr7Sv2nl5h70Nft577z70TM24"

#### We use paper environment for this example ####
paper = True  # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables

trade_api_url = None

In [2]:
import os

if api_key is None:
    api_key = os.environ.get('ALPACA_API_KEY')

if secret_key is None:
    secret_key = os.environ.get('ALPACA_SECRET_KEY')

In [3]:
# install alpaca-py if it is not available
try:
    import alpaca
except ImportError:
    !python3 -m pip install alpaca-py
    import alpaca

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 2.5 MB/s eta 0:00:00


In [52]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import calendar

from alpaca.trading.client import TradingClient
from alpaca.data.requests import (
    OptionBarsRequest,
    OptionTradesRequest,
    OptionLatestQuoteRequest,
    OptionLatestTradeRequest,
    OptionSnapshotRequest,
    OptionChainRequest
)
from alpaca.trading.requests import (
    GetOptionContractsRequest,
    MarketOrderRequest,
    OptionLegRequest,
    ReplaceOrderRequest,
    LimitOrderRequest,
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderStatus,
    TimeInForce,
    OrderClass,
    ContractType
)
from alpaca.common.exceptions import APIError

In [6]:
# setup client
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [7]:
# First, let us see if the account is enabled for multi-leg options trading
acct = trade_client.get_account()
if acct.options_trading_level >= 3:
    print("Account is enabled for multi-leg options trading!")
else:
    print("Account is not enabled for multi-leg options trading!")

Account is enabled for multi-leg options trading!


In [124]:
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockLatestTradeRequest
# Initialize Stock Data Client
stock_client = StockHistoricalDataClient(api_key, secret_key)

# Create a request to get the latest trade price for TSLA
trade_request = StockLatestTradeRequest(symbol_or_symbols=["TSLA"])
latest_trade = stock_client.get_stock_latest_trade(trade_request)

# Extract TSLA's latest price
tsla_price = latest_trade["TSLA"].price
print(f"Current TSLA Price: ${tsla_price}")

Current TSLA Price: $354.1


In [128]:
from alpaca.data.historical.option import OptionHistoricalDataClient

# Request all available TSLA option contracts
req = GetOptionContractsRequest(
    underlying_symbols=["TSLA"],  # Filter for TSLA options
    status=AssetStatus.ACTIVE,    # Only fetch active contracts
    expiration_date_lte='2025-02-21',
    type=ContractType.CALL,
    limit=1000                     # Increase limit if needed
)

# Fetch contracts
contracts = trade_client.get_option_contracts(req)
contracts



{   'next_page_token': None,
    'option_contracts': [   {   'close_price': '306.8',
                                'close_price_date': datetime.date(2025, 2, 18),
                                'expiration_date': datetime.date(2025, 2, 21),
                                'id': '7cdbe5bc-295c-4d37-bbb7-149eee7dca22',
                                'name': 'TSLA Feb 21 2025 50 Call',
                                'open_interest': '359',
                                'open_interest_date': datetime.date(2025, 2, 14),
                                'root_symbol': 'TSLA',
                                'size': '100',
                                'status': <AssetStatus.ACTIVE: 'active'>,
                                'strike_price': 50.0,
                                'style': <ExerciseStyle.AMERICAN: 'american'>,
                                'symbol': 'TSLA250221C00050000',
                                'tradable': True,
                                'type': <Contrac

In [129]:

# ✅ Extract symbols and strike prices
strike_prices = {contract.symbol: contract.strike_price for contract in contracts.option_contracts}

# Sort contracts by strike price (ascending order)
sorted_contracts = sorted(strike_prices.items(), key=lambda x: x[1])

# Extract sorted symbols
option_symbols = [symbol for symbol, _ in sorted_contracts]
# ✅ Extract the option symbols correctly
#option_symbols = [contract.symbol for contract in contracts.option_contracts]
#strike_prices = {contract.symbol: contract.strike_price for contract in contracts.option_contracts}
#setup option historical data client
option_historical_data_client = OptionHistoricalDataClient(api_key, secret_key, trade_api_url)


# Split symbols into batches of 100
batch_size = 100
option_quotes = {}

for i in range(0, len(option_symbols), batch_size):
    batch = option_symbols[i:i + batch_size]  # Get a batch of 100 symbols
    req = OptionLatestQuoteRequest(symbol_or_symbols=batch)
    #req = OptionSnapshotRequest(symbol_or_symbols=batch)
    try:
        res = option_historical_data_client.get_option_latest_quote(req)
        #res = option_historical_data_client.get_option_snapshot(req)
        option_quotes.update(res)  # Merge results into main dictionary
    except Exception as e:
        print(f"Error fetching batch {i // batch_size + 1}: {e}")

# Print results
print(f"Retrieved quotes for {len(option_quotes)} option contracts.")
option_quotes
for symbol, strike_price in sorted_contracts:
    bid_price = option_quotes[symbol].bid_price if symbol in option_quotes else "N/A"
    ask_price = option_quotes[symbol].ask_price if symbol in option_quotes else "N/A"

    print(f"{symbol:<20} {strike_price:<15} {bid_price:<15} {ask_price:<15}")

Retrieved quotes for 211 option contracts.
TSLA250221C00050000  50.0            307.74          311.89         
TSLA250221C00055000  55.0            292.21          306.44         
TSLA250221C00060000  60.0            291.44          300.4          
TSLA250221C00065000  65.0            281.82          289.58         
TSLA250221C00070000  70.0            280.98          284.98         
TSLA250221C00075000  75.0            272.92          282.14         
TSLA250221C00080000  80.0            271.87          276.78         
TSLA250221C00085000  85.0            262.51          268.25         
TSLA250221C00090000  90.0            260.56          270.58         
TSLA250221C00095000  95.0            257.1           262.58         
TSLA250221C00100000  100.0           250.09          253.46         
TSLA250221C00105000  105.0           249.53          250.82         
TSLA250221C00110000  110.0           244.81          249.31         
TSLA250221C00115000  115.0           239.87          243.97 